In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from arch import arch_model
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# --- Configuration v3.0 ---
RISK_FREE_RATE = 0.045  # 4.5%
TRADING_DAYS = 252
COMMISSION_PER_CONTRACT = 0.65 # $0.65 per contract
SLIPPAGE_PCT = 0.02 # 2% slippage

# --- Helper Functions v3.0 ---

def binomial_tree_price(S, K, T, r, sigma, N=100, option_type='call'):
    """
    Calculate American option price using Binomial Tree (CRR model).
    Handles early exercise.
    """
    if T <= 0:
        return max(0, S - K) if option_type == 'call' else max(0, K - S)
    
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    
    # Initialize asset prices at maturity
    asset_prices = np.zeros(N + 1)
    for i in range(N + 1):
        asset_prices[i] = S * (u ** (N - i)) * (d ** i)
        
    # Initialize option values at maturity
    option_values = np.zeros(N + 1)
    for i in range(N + 1):
        if option_type == 'call':
            option_values[i] = max(0, asset_prices[i] - K)
        else:
            option_values[i] = max(0, K - asset_prices[i])
            
    # Step back through the tree
    for j in range(N - 1, -1, -1):
        for i in range(j + 1):
            asset_prices[i] = S * (u ** (j - i)) * (d ** i)
            continuation_value = np.exp(-r * dt) * (p * option_values[i] + (1 - p) * option_values[i + 1])
            
            # Check for early exercise (American Option)
            if option_type == 'call':
                exercise_value = max(0, asset_prices[i] - K)
            else:
                exercise_value = max(0, K - asset_prices[i])
                
            option_values[i] = max(continuation_value, exercise_value)
            
    return option_values[0]

def get_vol_skew(atm_vol, strike, spot):
    """
    Simulate Volatility Skew (Smirk).
    """
    moneyness = strike / spot
    skew_factor = 1.0
    
    if moneyness < 1.0: # OTM Put / ITM Call
        skew_factor = 1.0 + (1.0 - moneyness) * 0.5 
    else: # OTM Call / ITM Put
        skew_factor = 1.0 + (moneyness - 1.0) * 0.2
        
    return atm_vol * skew_factor

def apply_transaction_costs(price, contracts=1):
    """
    Add slippage and commission.
    """
    buy_price = price * (1 + SLIPPAGE_PCT) + (COMMISSION_PER_CONTRACT / 100)
    sell_price = price * (1 - SLIPPAGE_PCT) - (COMMISSION_PER_CONTRACT / 100)
    return buy_price, sell_price

# --- Advanced Volatility Modeling (GARCH + VIX) ---

def get_garch_volatility(history):
    """
    Estimate Conditional Volatility using GARCH(1,1).
    Returns the annualized volatility for the last day.
    """
    returns = 100 * np.log(history['Close'] / history['Close'].shift(1)).dropna()
    
    if len(returns) < 30:
        # Fallback to simple std dev if not enough data
        return returns.std() / 100 * np.sqrt(TRADING_DAYS)
        
    try:
        model = arch_model(returns, vol='Garch', p=1, q=1)
        res = model.fit(disp='off')
        # Forecast next day volatility
        forecast = res.forecast(horizon=1)
        daily_vol = np.sqrt(forecast.variance.iloc[-1, 0])
        annualized_vol = (daily_vol / 100) * np.sqrt(TRADING_DAYS)
        return annualized_vol
    except:
        # Fallback
        return returns.std() / 100 * np.sqrt(TRADING_DAYS)

def fetch_vix_data(start_date, end_date):
    """Fetch VIX data for scaling."""
    try:
        vix = yf.Ticker("^VIX")
        hist = vix.history(start=start_date, end=end_date)
        return hist['Close']
    except:
        return None

def fetch_data(ticker_symbol, start_date, end_date):
    """Fetch historical data from yfinance."""
    ticker = yf.Ticker(ticker_symbol)
    hist = ticker.history(start=start_date, end=end_date)
    return hist

# --- Strategy 1: Analyst Day Volatility Arbitrage ---

def run_analyst_day_strategy(vix_data):
    print("\n--- Strategy 1: Analyst Day Volatility Arbitrage (HOOD Example) ---")
    ticker_symbol = "HOOD"
    event_date = "2024-12-04"
    
    event_dt = datetime.strptime(event_date, "%Y-%m-%d")
    entry_date = (event_dt - timedelta(days=7)).strftime("%Y-%m-%d") # T-5
    exit_date = (event_dt + timedelta(days=1)).strftime("%Y-%m-%d")  # T+1
    
    # Fetch Data
    data_start = (event_dt - timedelta(days=365)).strftime("%Y-%m-%d") # Need more history for GARCH
    data_end = (event_dt + timedelta(days=5)).strftime("%Y-%m-%d")
    hist = fetch_data(ticker_symbol, data_start, data_end)
    
    if hist.empty: return None

    try:
        entry_row = hist.asof(entry_date)
        exit_row = hist.asof(exit_date)
    except: return None
        
    spot_entry = entry_row['Close']
    strike = np.ceil(spot_entry) 
    
    # IV: GARCH + VIX Scaling
    # Calculate GARCH vol up to entry date
    hist_entry = hist.loc[:entry_date]
    garch_vol = get_garch_volatility(hist_entry)
    
    # VIX Scaling
    # If VIX is high, boost IV.
    # Baseline VIX ~ 20.
    current_vix = 20.0
    if vix_data is not None:
        try:
            current_vix = vix_data.asof(entry_date)
        except: pass
    
    vix_scalar = max(1.0, current_vix / 20.0)
    atm_vol = garch_vol * vix_scalar
    
    iv_entry = get_vol_skew(atm_vol, strike, spot_entry)
    
    expiry_date = "2024-12-06"
    expiry_dt = datetime.strptime(expiry_date, "%Y-%m-%d")
    entry_dt = datetime.strptime(entry_date, "%Y-%m-%d")
    dte_entry = (expiry_dt - entry_dt).days / 365.0
    
    # Binomial Pricing
    mid_price_entry = binomial_tree_price(spot_entry, strike, dte_entry, RISK_FREE_RATE, iv_entry, option_type='call')
    buy_price_entry, _ = apply_transaction_costs(mid_price_entry)
    
    # Exit
    spot_exit = exit_row['Close']
    exit_dt = datetime.strptime(exit_date, "%Y-%m-%d")
    dte_exit = (expiry_dt - exit_dt).days / 365.0
    
    iv_exit = iv_entry # Assume constant vol for simplicity (or crush)
    mid_price_exit = binomial_tree_price(spot_exit, strike, dte_exit, RISK_FREE_RATE, iv_exit, option_type='call')
    _, sell_price_exit = apply_transaction_costs(mid_price_exit)
    
    pnl = (sell_price_exit - buy_price_entry) / buy_price_entry
    
    print(f"Entry: ${buy_price_entry:.2f} (Mid: {mid_price_entry:.2f}, IV: {iv_entry:.2%})")
    print(f"Exit: ${sell_price_exit:.2f} (Mid: {mid_price_exit:.2f})")
    print(f"P&L: {pnl:.2%}")
    
    return {
        "Strategy": "Analyst Day",
        "Ticker": ticker_symbol,
        "Entry Date": entry_date,
        "Return": pnl
    }

# --- Strategy 2: Pre-Earnings Call Buying ---

def run_earnings_strategy(vix_data, tickers=['BLK', 'MSFT', 'NVDA', 'AAPL', 'AMD', 'TSLA', 'META', 'AMZN']):
    print("\n--- Strategy 2: Pre-Earnings Call Buying ---")
    results = []
    
    for ticker_symbol in tickers:
        try:
            ticker = yf.Ticker(ticker_symbol)
            earnings = ticker.get_earnings_dates(limit=12)
            if earnings is None or earnings.empty: continue
                
            now_aware = datetime.now().astimezone()
            past_earnings = earnings.index[earnings.index < now_aware]
            
            for event_dt in past_earnings[:2]:
                event_date = event_dt.strftime("%Y-%m-%d")
                entry_dt = event_dt - timedelta(days=4) # T-2
                entry_date = entry_dt.strftime("%Y-%m-%d")
                exit_dt = event_dt + timedelta(days=1) # T+1
                exit_date = exit_dt.strftime("%Y-%m-%d")
                
                data_start = (entry_dt - timedelta(days=365)).strftime("%Y-%m-%d")
                data_end = (exit_dt + timedelta(days=5)).strftime("%Y-%m-%d")
                hist = fetch_data(ticker_symbol, data_start, data_end)
                
                if hist.empty: continue
                hist.index = hist.index.tz_localize(None)
                
                try:
                    entry_row = hist.asof(entry_date)
                    exit_row = hist.asof(exit_date)
                except: continue
                    
                if pd.isna(entry_row['Close']) or pd.isna(exit_row['Close']): continue

                spot_entry = entry_row['Close']
                strike = np.ceil(spot_entry * 1.02) # 2% OTM
                
                # IV: GARCH + VIX + Earnings Premium
                hist_entry = hist.loc[:entry_date]
                garch_vol = get_garch_volatility(hist_entry)
                
                current_vix = 20.0
                if vix_data is not None:
                    try: current_vix = vix_data.asof(entry_date)
                    except: pass
                vix_scalar = max(1.0, current_vix / 20.0)
                
                atm_vol = garch_vol * vix_scalar * 1.5 # Earnings Premium
                iv_entry = get_vol_skew(atm_vol, strike, spot_entry)
                
                dte_entry = 10 / 365.0
                mid_price_entry = binomial_tree_price(spot_entry, strike, dte_entry, RISK_FREE_RATE, iv_entry, option_type='call')
                buy_price_entry, _ = apply_transaction_costs(mid_price_entry)
                
                # Exit
                spot_exit = exit_row['Close']
                dte_exit = (10 - 3) / 365.0
                
                # IV Crush
                iv_exit = (garch_vol * vix_scalar) * 1.0 # Crush back to normal (no premium)
                mid_price_exit = binomial_tree_price(spot_exit, strike, dte_exit, RISK_FREE_RATE, iv_exit, option_type='call')
                _, sell_price_exit = apply_transaction_costs(mid_price_exit)
                
                pnl = (sell_price_exit - buy_price_entry) / buy_price_entry
                
                results.append({
                    "Strategy": "Earnings",
                    "Ticker": ticker_symbol,
                    "Event Date": event_date,
                    "Return": pnl
                })
                print(f"{ticker_symbol} Earnings {event_date}: {pnl:.2%}")
                
        except Exception as e:
            print(f"Error {ticker_symbol}: {e}")
            
    return results

# --- Strategy 3: Asymmetric Alpha (Covered Call) ---

def run_covered_call_strategy(vix_data, tickers=['AAPL', 'MSFT', 'GOOGL', 'JNJ', 'PG', 'KO', 'PEP', 'XOM']):
    print("\n--- Strategy 3: Asymmetric Alpha (Covered Call) ---")
    results = []
    start_date = (datetime.now() - timedelta(days=365*2)).strftime("%Y-%m-%d") # 2 Years for GARCH
    end_date = datetime.now().strftime("%Y-%m-%d")
    
    for ticker_symbol in tickers:
        hist = fetch_data(ticker_symbol, start_date, end_date)
        hist.index = hist.index.tz_localize(None)
        dates = hist.index[::21]
        
        # Only simulate last 12 months
        dates = dates[dates > (datetime.now() - timedelta(days=365))]
        
        for i in range(len(dates)-1):
            entry_date = dates[i]
            exit_date = dates[i+1]
            
            spot_entry = hist.loc[entry_date]['Close']
            strike = spot_entry * 1.10 # 10% OTM
            
            # IV: GARCH + VIX
            hist_entry = hist.loc[:entry_date]
            garch_vol = get_garch_volatility(hist_entry)
            
            current_vix = 20.0
            if vix_data is not None:
                try: current_vix = vix_data.asof(entry_date)
                except: pass
            vix_scalar = max(1.0, current_vix / 20.0)
            
            atm_vol = garch_vol * vix_scalar
            iv_entry = get_vol_skew(atm_vol, strike, spot_entry)
            
            dte = 30 / 365.0
            mid_price = binomial_tree_price(spot_entry, strike, dte, RISK_FREE_RATE, iv_entry, option_type='call')
            _, sell_price = apply_transaction_costs(mid_price) # Selling the call
            
            spot_exit = hist.loc[exit_date]['Close']
            
            call_value_exit = max(0, spot_exit - strike)
            cost_to_close = call_value_exit
            if call_value_exit > 0:
                cost_to_close += (COMMISSION_PER_CONTRACT / 100)
            
            option_pnl = sell_price - cost_to_close
            total_pnl = (spot_exit - spot_entry + option_pnl) / spot_entry
            
            results.append({
                "Strategy": "Covered Call",
                "Ticker": ticker_symbol,
                "Period": f"{entry_date.date()}",
                "Return": total_pnl
            })
            
    avg_ret = np.mean([r['Return'] for r in results])
    print(f"Average Monthly Return: {avg_ret:.2%}")
    return results

if __name__ == "__main__":
    # Fetch VIX Data once
    print("Fetching VIX data...")
    vix_data = fetch_vix_data("2023-01-01", datetime.now().strftime("%Y-%m-%d"))
    if vix_data is not None:
        vix_data.index = vix_data.index.tz_localize(None)
    
    results = []
    
    res1 = run_analyst_day_strategy(vix_data)
    if res1: results.append(res1)
    results.extend(run_earnings_strategy(vix_data))
    results.extend(run_covered_call_strategy(vix_data))
    
    print("\n" + "="*50)
    print("GOLDMAN SACHS STRATEGIES: BACKTEST REPORT v3.0")
    print("="*50)
    
    df_res = pd.DataFrame(results)
    summary = df_res.groupby("Strategy")['Return'].agg(['count', 'mean', 'min', 'max'])
    summary.columns = ['Trades', 'Avg Return', 'Min Return', 'Max Return']
    summary['Win Rate'] = df_res.groupby("Strategy")['Return'].apply(lambda x: (x > 0).mean())
    
    print("\n## 1. Performance Summary")
    print(summary.to_markdown(floatfmt=".2%"))
    
    print("\n## 2. Trade Log")
    print(df_res[['Strategy', 'Ticker', 'Return']].to_markdown(index=False, floatfmt=".2%"))
    
    with open("backtest_report_v3.md", "w") as f:
        f.write("# Goldman Sachs Strategies Backtest Report v3.0 (GARCH+VIX)\n\n")
        f.write("## 1. Performance Summary\n")
        f.write(summary.to_markdown(floatfmt=".2%"))
        f.write("\n\n## 2. Trade Log\n")
        f.write(df_res[['Strategy', 'Ticker', 'Return']].to_markdown(index=False, floatfmt=".2%"))
    print("\n[+] Report saved to 'backtest_report_v3.md'")



Fetching VIX data...

--- Strategy 1: Analyst Day Volatility Arbitrage (HOOD Example) ---

--- Strategy 2: Pre-Earnings Call Buying ---
BLK Earnings 2025-04-11: -5.12%
BLK Earnings 2025-01-15: 128.92%
MSFT Earnings 2025-04-30: 142.35%
MSFT Earnings 2025-01-29: -100.07%
NVDA Earnings 2025-06-26: 123.78%
NVDA Earnings 2025-05-28: 18.35%
AAPL Earnings 2025-05-01: -76.39%
AAPL Earnings 2025-02-27: -87.77%
AMD Earnings 2025-05-08: -8.77%
AMD Earnings 2025-05-06: -39.87%
TSLA Earnings 2025-04-22: -32.47%
TSLA Earnings 2025-01-29: -62.79%
META Earnings 2025-05-29: -8.38%
META Earnings 2025-04-30: -17.41%
AMZN Earnings 2025-05-22: -77.30%
AMZN Earnings 2025-05-01: -50.27%

--- Strategy 3: Asymmetric Alpha (Covered Call) ---
Average Monthly Return: 1.27%

GOLDMAN SACHS STRATEGIES: BACKTEST REPORT v3.0

## 1. Performance Summary
| Strategy     |   Trades |   Avg Return |   Min Return |   Max Return |   Win Rate |
|:-------------|---------:|-------------:|-------------:|-------------:|-----------